## Entornos RL

In [1]:
# HIDDEN

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### ¿Qué es un entorno?

- Un entorno puede ser
  - un juego, como un videojuego
  - una simulación de un escenario del mundo real, como un robot, el comportamiento de un usuario o el mercado de valores
  - cualquier otra configuración con un _agente_ que realiza _acciones_, ve _observaciones_ y recibe _recompensas_
  
Nota sobre la terminología: utilizaremos indistintamente _agente_ y _jugador_ 

#### Ejemplo de carrera: lago congelado

Como ejemplo de ejecución de un entorno, utilizaremos el entorno [Frozen Lake](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/) de [OpenAI Gym](https://www.gymlibrary.ml/), que proporciona la interfaz estándar para los problemas de RL. Podemos visualizar el entorno así:

In [2]:
import gym
env = gym.make("FrozenLake-v1", is_slippery=False)

In [3]:
# HIDDEN
from utils_01 import fix_frozen_lake_render
fix_frozen_lake_render(env)

In [4]:
env.reset()
env.render()


P...
.O.O
...O
O..G


El objetivo es que el jugador (`P`) llegue a la meta (`G`) caminando por los segmentos del lago helado (`.`) sin caer en los agujeros (`O`).

**Nota**: el renderizado del entorno se ha modificado con respecto al renderizado del Gimnasio OpenAI para que pueda mostrarse con claridad en esta plataforma de aprendizaje interactivo.

Notas:

El código no se muestra para la sustitución del renderizado.

#### Movimiento

El jugador puede moverse por el lago helado. Por ejemplo:

In [5]:
env.step(1) # 1 -> Down
env.render()

  (Down)
....
PO.O
...O
O..G


No te preocupes por `paso(1)` por ahora; ya llegaremos a eso 

Lo que puedes ver es que el jugador (`P`) se movió hacia abajo.

#### Objetivo

Avanza un montón de pasos y habrás completado el puzzle:

In [6]:
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.render()

  (Right)
....
.O.O
...O
O..P


Has alcanzado el objetivo al llegar a la parte inferior derecha.

#### ¿Qué hace un entorno?

Un entorno implica varios componentes clave, que repasaremos en las siguientes diapositivas.

#### Estados

- Utilizaremos el término _estado_ de manera informal para referirnos a todo lo relacionado con el entorno 

In [7]:
env.reset()
env.render()


P...
.O.O
...O
O..G


- Por ejemplo, este es el estado inicial del entorno.
- El jugador está en la parte superior izquierda, hay hielo congelado cerca, etc.
- Utilizaremos el concepto de estado para hablar de nuestro entorno, pero no aparecerá en la "API".

#### Acciones

- Aquí, el jugador puede elegir entre 4 acciones posibles: izquierda, abajo, derecha, arriba
- El espacio de todas las acciones posibles se denomina **espacio de acción**.
- En SL distinguimos entre regresión (continua $y$) y clasificación (categórica $y$)
- Del mismo modo, en RL el espacio de acción puede ser continuo o discreto
- En este caso, es discreto (4 posibilidades)
- El código está de acuerdo:

In [8]:
env.action_space

Discrete(4)

#### Observaciones

- Las observaciones son las _partes del estado que el agente puede ver_.
- A veces, el agente puede verlo todo; a esto lo llamamos _completamente observable_.
- A menudo, tenemos entornos _parcialmente observables_ 
- En el ejemplo del Lago Helado, el agente sólo puede ver su propia ubicación de las 16 casillas.
- Al agente no se le "dice" dónde están los agujeros mediante observaciones directas, por lo que tendrá que _aprender_ esto mediante ensayo y error.

#### Observaciones

- El espacio de todas las observaciones posibles se denomina **espacio de observación**.
- Puedes pensar en el espacio de acción como análogo al objetivo en el aprendizaje supervisado.
- Puedes pensar en el espacio de observación como análogo a las características en el aprendizaje supervisado.


Aquí tenemos un espacio de observación discreto formado por las 16 posiciones posibles del jugador:

In [9]:
env.observation_space

Discrete(16)

#### Recompensas

- En el aprendizaje supervisado, el objetivo suele ser hacer buenas predicciones.
- Puedes probar diferentes funciones de pérdida según tu objetivo específico, pero el concepto general es el mismo.
- En RL, el objetivo puede ser cualquier cosa.
- Pero, al igual que en la SL, tendrás que _optimizar_ algo.
- En RL, nuestro objetivo es maximizar la **recompensa**.

#### Recompensas 

En el ejemplo del Lago Helado, el agente recibe una recompensa cuando alcanza el objetivo.

In [10]:
env.reset()
obs, reward, done, _ = env.step(0)
env.render()
print("reward =", reward)

  (Left)
P...
.O.O
...O
O..G
reward = 0.0


In [11]:
obs, reward, done, _ = env.step(1)
print("reward =", reward)

reward = 0.0


Todavía no hay recompensa, sigamos...

#### Recompensas

In [12]:
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.step(1)
obs, reward, done, _ = env.step(2)
env.render()
print("reward =", reward)

  (Right)
....
.O.O
...O
O..P
reward = 1.0


Obtuvimos una recompensa de 1,0 por alcanzar el objetivo.

#### Bucle ambiente-agente

Observa cómo el agente (en este caso, nosotros) y el entorno se comunican en un bucle de ida y vuelta:

![](img/RL-loop.png)

Este es el clásico diagrama que verás en todas partes.

El objetivo será aprender automáticamente el comportamiento del agente (¡permanece atento!).

#### Representación de acciones

- Para utilizar el software de RL, necesitaremos una representación numérica de nuestro espacio de acción y de nuestro espacio de observación.
- En este caso, tenemos 4 posibles acciones discretas, por lo que podemos codificarlas como {0,1,2,3} para (izquierda, abajo, derecha, arriba).
- Por eso, antes hicimos

In [13]:
env.step(1)

para caminar hacia abajo.

#### Representar las observaciones

- Del mismo modo, necesitaremos una representación numérica de nuestras observaciones.
- Aquí hay 16 posiciones posibles del jugador. Éstas se codifican de 0 a 15 de la siguiente manera:

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

Estos detalles del entorno de Frozen Lake también están disponibles en la [documentación](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/).

#### Representar las observaciones

Inicialmente, observamos "0" porque empezamos en la parte superior izquierda:

In [14]:
env.reset()
env.render()


P...
.O.O
...O
O..G


Después de desplazarnos hacia abajo (acción 1), pasamos a la posición 4.

In [15]:
obs, reward, done, _ = env.step(1)
obs

4

La observación es devuelta por el método `step()`.

#### Entornos no deterministas

- Hasta ahora, realizar una acción determinada desde un estado concreto siempre daba como resultado el mismo estado nuevo.
- En otras palabras, nuestro entorno del Lago Helado era _determinista_.
- Algunos entornos son _no deterministas_, lo que significa que el resultado de una acción puede ser aleatorio.
- Podemos inicializar un Lago Congelado no determinista así

In [16]:
env_slippery = gym.make("FrozenLake-v1", is_slippery=True)

In [17]:
# HIDDEN
env_slippery.seed(4)
fix_frozen_lake_render(env_slippery)

In [18]:
env_slippery.reset()
env_slippery.render()


P...
.O.O
...O
O..G


#### Entornos no deterministas

In [19]:
env_slippery.step(1) # move down
env_slippery.render()

  (Down)
....
PO.O
...O
O..G


La mudanza hacia abajo no funcionó como estaba previsto.

In [20]:
env_slippery.step(1) # move down
env_slippery.render()

  (Down)
....
.P.O
...O
O..G


Esta vez ha funcionado el movimiento hacia abajo.

En este entorno "resbaladizo" del Lago Helado, el movimiento sólo funciona como se pretende 1/3 de las veces.

#### Episodios

- Jugar al Lago Helado tiene un final: o bien caes en un agujero o llegas a la meta.
- Sin embargo, una sola partida no es suficiente para que el algoritmo de RL aprenda.
- Necesitará varias partidas, llamadas **episodios**.
- Después de un episodio, el entorno se reinicia.

#### Episodios

El método `paso()` devuelve una bandera que nos indica si el episodio ha terminado:

In [21]:
obs, reward, done, _ = env_slippery.step(1)
done

True

In [22]:
env_slippery.render()

  (Down)
....
.P.O
...O
O..G


Aquí el episodio ha terminado porque hemos caído en un agujero.

#### Episodios

- En algunos entornos (como el Lago Helado), las recompensas sólo se reciben al final de un episodio.
- En otros entornos, las recompensas pueden recibirse en cualquier **paso de tiempo** (es decir, después de una acción).

#### Ponerlo todo junto

- Ya hemos hablado de los principales componentes de un entorno RL:
  - Estados
  - Acciones
  - Observaciones
  - Recompensas
  - Episodios
  

#### conjuntos de datos de SL frente a entornos de RL

- En el aprendizaje supervisado, normalmente se te da un conjunto de datos.
- En RL, el entorno actúa como un _generador de datos_.
  - Cuanto más juegues con el entorno, más "datos" generarás y más podrás aprender.
- También se puede hacer RL con un conjunto de datos previamente recogidos (lo que se llama _RL offline_), pero eso está fuera de nuestro alcance.

#### ¡Apliquemos lo que hemos aprendido!

## Entorno del coche autodirigido
<!-- multiple choice -->

Estás utilizando RL para entrenar un coche de autoconducción. La IA del coche utiliza varias cámaras y sensores como entradas y tiene que decidir el ángulo del volante, así como el ángulo de los pedales de gas/freno en el suelo.

#### ¿Es el espacio de observación continuo o discreto?

- [x] Continuo
- [ ] Discreto | En este caso, las observaciones son las entradas de los sensores, por ejemplo, las estimaciones de profundidad.

#### ¿Es el espacio de acción continuo o discreto?

- [x] Continuo
- [ ] Discreto | Las acciones son ángulos; no proceden de un conjunto discreto de opciones.

#### ¿Cuál sería la estructura de recompensa más razonable para este entorno?

- [] La recompensa es igual a la cantidad de tiempo que el coche fue capaz de conducir sin chocar | ¿Cuál sería la recompensa si el coche no se mueve nunca?
- [x] La recompensa es igual a la distancia que el coche fue capaz de recorrer sin chocar | ¡Sí, eso suena bien!
- [ ] +1 de recompensa cada vez que el coche se estrelle | Ten en cuenta que queremos maximizar la recompensa, no minimizarla.

## Episodios frente a pasos de tiempo
<!-- multiple choice -->

Completa los espacios en blanco de la siguiente frase 

*En un entorno de aprendizaje por refuerzo, se realizan acciones de forma repetida hasta que el \_\_\_\_termina. Puede tratarse de una sola \N acción, o de muchas.*

- [ ] paso de tiempo / recompensa | ¡Comprueba cuidadosamente el primer espacio en blanco!
- [ ] paso de tiempo / recompensa | ¡Inténtalo de nuevo!
- [ ] paso de tiempo / episodio | ¡Inténtalo de nuevo!
- [x] episodio / paso de tiempo | ¡Lo tienes!


## Entorno de taxi del gimnasio
<!-- coding exercise -->

En este ejercicio veremos uno de los entornos basados en texto incluidos en
OpenAI gym, llamado entorno taxi.
Documentación [aquí](https://www.gymlibrary.ml/environments/toy_text/taxi/).

In [33]:
# EXERCISE
import gym

taxi = gym.make("Taxi-v3")
obs = taxi.reset(seed=5)
taxi.render()

# Add calls to `step` here!

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



En este ejercicio, el taxi está representado por el resaltado amarillo,
actualmente en la parte inferior izquierda de la arena. El `:` se puede cruzar, pero el `|` no.
El objetivo es recoger pasajeros y dejarlos.

Hay 6 acciones posibles:
abajo (0), arriba (1), derecha (2), izquierda (3), recoger (4), dejar (5).

In [36]:
taxi.action_space

Discrete(6)

El siguiente código imprime la observación en un formato más legible para los humanos:

In [37]:
print("Taxi row: %d\nTaxi col: %d\nPassenger loc: %d\nDestination loc: %d" % tuple(taxi.decode(obs)))

Taxi row: 4
Taxi col: 0
Passenger loc: 0
Destination loc: 2


Las posibles ubicaciones son `R` (0), `G` (1), `Y` (2), `B` (3) y en taxi (4). Así, el pasajero se encuentra actualmente en `R` y se dirige a `Y`.

Para responder a la siguiente pregunta, tendrás que modificar el código, ejecutarlo e imprimir cualquier resultado relevante.

In [42]:
# SOLUTION
import gym

taxi = gym.make("Taxi-v3")
obs = taxi.reset(seed=5)
taxi.render()
taxi.step(1)
taxi.step(1)
taxi.step(1)
taxi.step(1)
taxi.render()
taxi.step(4)
taxi.step(0)
taxi.step(0)
taxi.step(0)
taxi.step(0)
taxi.render()
obs, reward, done, _ = taxi.step(5)
print(reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
20


In [27]:
# TODO: in this case there is no testing of the code; the code is only used to explore
# this may not work well with the framework - add an automated test to the code as well? 
# e.g. can check that the state of the env is correct?

#### ¿Qué recompensa recibe el agente por dejar al pasajero con éxito?

- [ ] 0 | Intenta realizar las acciones necesarias con `taxi.step()` e imprimir las recompensas.
- [5 Intenta realizar las acciones necesarias con "taxi.step()` e imprimir las recompensas.
- [10 Intenta realizar las acciones necesarias con "taxi.step()` e imprimir las recompensas.
- [x] 20 | ¡Lo tienes!

## Observaciones frente a renders
<!-- coding exercise -->

El entorno del lago congelado nos permite crear una representación visual del entorno, que hemos visto antes. Esto es para fines humanos/depuración, y _no_ lo ve el agente/algoritmo. Tu tarea aquí es reproducir un entorno dado de Frozen Lake **sin mirar la representación visual** (¡sin hacer trampas!). Rellena la lista de "acciones" para que contenga un conjunto de acciones que lleven correctamente al agente a la meta. ¡Lo que estás experimentando es lo que un algoritmo de RL "ve" cuando aprende!

Ten en cuenta que el entorno de Frozen Lake dado es de 3x3 en lugar de 4x4. Por tanto, el espacio de observación va de 0 a 8 en lugar de 0 a 15.

In [28]:
# EXERCISE

import gym
import numpy as np
np.random.seed(1)
env = gym.make("FrozenLake-v1", 
               desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), 
               is_slippery=False)
env.render = None

obs = env.reset()
actions = ____
for action in actions:
    obs, reward, done, _ = env.step(action)
    print("Obs:", obs, "Reward:", reward, "Done:", done)


NameError: name '____' is not defined

In [6]:
# SOLUTION

import gym.envs.toy_text
import numpy as np
np.random.seed(1)
env = gym.make("FrozenLake-v1", desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), is_slippery=False)
env.render = None

obs = env.reset()
actions = []
# BEGIN SOLUTION
actions = [1,2,1,2]
# END SOLUTION
for action in actions:
    obs, reward, done, _ = env.step(action)
    print("Obs:", obs, "Reward:", reward, "Done:", done)

Obs: 3 Reward: 0.0 Done: False
Obs: 4 Reward: 0.0 Done: False
Obs: 7 Reward: 0.0 Done: False
Obs: 8 Reward: 1.0 Done: True


In [9]:
# HIDDEN
import numpy as np
import gym.envs.toy_text
from utils_01 import fix_frozen_lake_render
np.random.seed(1)
env = gym.make("FrozenLake-v1", desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), is_slippery=False)
env.reset()
fix_frozen_lake_render(env)
env.render()


PO.
...
O.G


#### El mejor camino hacia la meta

Recordando que las acciones 0, 1, 2, 3 representan izquierda, abajo, derecha, arriba (respectivamente), ¿cuál de las siguientes afirmaciones describe correctamente el mejor camino hacia la meta?

- [ ] Empieza moviéndote hacia abajo, y luego hacia abajo de nuevo
- [x] Empieza moviéndote hacia abajo, y luego hacia la derecha
- [ ] Empieza moviéndote a la derecha, luego a la derecha otra vez
- [ ] Empieza moviéndote hacia la derecha y luego hacia abajo